In [4]:
import pandas as pd
import numpy as np

In [1]:
import streamlit as st

2022-12-23 19:15:08.777 INFO    numexpr.utils: Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-12-23 19:15:08.777 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
# Прошу пользователя загрузить фаил
uploaded_file = st.file_uploader(label='Фаил')

2022-12-23 19:16:19.983 
  command:

    streamlit run C:\Users\79109\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [ ]:
# Когда он его загрузит, я сохраняю его в переменной uploaded_file
uploaded_file = pd.read_csv(uploaded_file, sep=';').drop(columns=['<DATE>', '<TIME>'])

In [84]:
data1 = pd.read_csv('btc1.csv', sep=';').drop(columns=['<DATE>', '<TIME>'])
data2 = data1[:5]
data2.tail()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,13863.14,13889.0,12952.50,13480.01,5208
1,13480.00,15275.0,13005.01,14781.51,10444
2,14781.52,15400.0,14628.00,15098.14,8382
3,15098.23,15400.0,14230.00,15144.99,9077
4,15145.00,17178.0,14824.05,16960.01,9286


In [85]:
# Таблица с удаленными колонками для y_scaler
data_y_scaler = data2.drop(columns=['<OPEN>', '<HIGH>', '<LOW>', '<VOL>'])
data_y_scaler

,<CLOSE>
0,13480.01
1,14781.51
2,15098.14
3,15144.99
4,16960.01


In [86]:
x_data = data2.to_numpy()

# Проверка данных
print(x_data.shape)
print(type(x_data))

(5, 5)
<class 'numpy.ndarray'>


In [87]:
from sklearn.preprocessing import StandardScaler

In [88]:
# Масштабирование данных
x_scaler = StandardScaler()
x_scaler.fit(x_data)
x_data_train = x_scaler.transform(x_data)

# Проверка формы данных
print(f'Train data: {x_data_train.shape}')

Train data: (5, 5)


In [89]:
# Масштабирование данных
y_scaler = StandardScaler()
y_scaler.fit(data_y_scaler)
y_data_train = y_scaler.transform(data_y_scaler)

# Проверка формы данных
print(f'Predict data: {y_data_train.shape}')

Predict data: (5, 1)


In [90]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


In [91]:
generator = TimeseriesGenerator(x_data_train,
                                x_data_train,
                                length=4,
                                batch_size=1)

# Формирование тестовой выборки из генератора
x_predict, y_test = generator[0]

# Проверка формы тестовой выборки
print(f'x_predict: {x_predict.shape}, y: {y_test.shape}')
x_predict

x_predict: (1, 4, 5), y: (1, 5)


array([[[-0.90089255, -1.47366719, -1.22163031, -1.44994204,
         -1.85331772],
        [-1.46633568, -0.14684978, -1.15586548, -0.27995393,
          1.11298771],
        [ 0.45446492, -0.02718731,  0.87680767,  0.00468175,
         -0.05517917],
        [ 0.92186977, -0.02718731,  0.37834254,  0.04679773,
          0.33855312]]])

In [92]:
y_test

array([[0.99089356, 1.67489159, 1.12234558, 1.67841649, 0.45695606]])

In [93]:
from tensorflow.keras.models import load_model

In [94]:
model=load_model('btc_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4, 1024)           6144      
                                                                 
 dense (Dense)               (None, 4, 16)             16400     
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 520       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 23,073
Trainable params: 23,073
Non-trainable params: 0
_________________________________________________________________


In [95]:
model_predict = y_scaler.inverse_transform(model.predict(x_predict))

print(f'Цена биткоина завтра {model_predict}')

1/1 [==============================] - 0s 114ms/step
Цена биткоина завтра [[15073.242]]


In [ ]:
st.title('Новая улучшенная классификации изображений в облаке Streamlit')
img = load_image()
result = st.button('Распознать изображение')
if result:
    x = preprocess_image(img)
    preds = model.predict(x)
    st.write('**Результаты распознавания:**')
    print_predictions(preds)